# Домашнее задание «Продвинутый pandas 2»

In [1]:
import pandas as pd

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [116]:
lifetime = (pd
              .DataFrame(
                  {'lifetime' : (ratings.groupby('userId').max()['timestamp'] - ratings.groupby('userId').min()['timestamp'])}
                        )
              .reset_index())
lifetime

,userId,lifetime
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101
...,...,...
666,667,1014
667,668,282
668,669,685
669,670,2162705


In [117]:
regular_users = pd.DataFrame({'userId' :  [ i
                                            for i in range(1, 672) 
                                            if len(ratings[ratings['userId'] == i]) > 100
                                          ]
                             })

regular_users

,userId
0,4
1,8
2,15
3,17
4,19
...,...
253,656
254,659
255,664
256,665


In [125]:
result = regular_users.merge(lifetime, on='userId', how='left')
result

,userId,lifetime
0,4,203560
1,8,85187
2,15,471393496
3,17,8053
4,19,5282
...,...,...
253,656,3053
254,659,31609411
255,664,98180439
256,665,54131471


In [137]:
print(f"Среднее время жизни — {result['lifetime'].mean() / 60 / 60 / 24} дня")

Среднее время жизни — 463.89476214829165 дня


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).  
Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [16]:
table_1 = rzd.merge(auto.merge(air, on = 'client_id', how = 'outer'), on = 'client_id', how = 'outer')
table_1.fillna(' ')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093,,
1,112,2810,,
2,113,10283,57483,
3,114,5774,83,
4,115,981,912,81
5,116,,4834,4
6,117,,98,13
7,118,,,173


In [15]:
table_2 = table_1.merge(client_base, on = 'client_id', how = 'outer') 
table_2.fillna(' ')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093,,,Комсомольская 4
1,112,2810,,,Энтузиастов 8а
2,113,10283,57483,,Левобережная 1а
3,114,5774,83,,Мира 14
4,115,981,912,81,ЗЖБИиДК 1
5,116,,4834,4,Строителей 18
6,117,,98,13,Панфиловская 33
7,118,,,173,Мастеркова 4
